In [ ]:
# add default values for parameters here

## Simple example comparing land variables from two simulations
Created by wwieder@ucar.edu, Jan 2024

- paths are harded coded for now
- quickly (and inaccurately) calcultes a 5 year mean from raw, .h0., files.
- plots global means and differences which  also not very nice to look at, especially since this points to a 4x5 grid.

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import time
import datetime
import calendar
import cftime

from glob import glob
from os.path import join

import dask

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.pyplot import cm
import cartopy.crs as ccrs

from distributed import wait

%matplotlib inline

In [ ]:
# -- read only these variables from the whole netcdf files
# average over time
def preprocess (ds):
    variables = ['TOTECOSYSC', 'TOTVEGC','TOTSOMC',
                 'SOM_PAS_C_vr']

    ds_new= ds[variables]
    return ds_new



In [ ]:
case = 'ctsm51d159_f45_GSWP3_bgccrop_'
type = ['1850pAD','1850pSASU']
cases = [case+'1850pAD',case+'1850pSASU']
cases

In [ ]:
for c in range(len(cases)):

    sim_files =[]
    sim_path = "/glade/campaign/cesm/development/cross-wg/diagnostic_framework/CESM_output_for_testing/"
    sim_path = sim_path+cases[c]+"/lnd/hist/"
    sim_files.extend(sorted(glob(join(sim_path+cases[c]+".clm2.h0.*.nc"))))
    # subset last 5 years of data 
    sim_files = sim_files[-60:None]
    print("All simulation files for "+cases[c]+": [", len(sim_files), "files]")

    temp = xr.open_mfdataset(sim_files, decode_times=True, combine='by_coords',
                             parallel=True,preprocess=preprocess).mean('time')
  
    if c == 0:
        ds = temp
    else:
        ds = xr.concat([ds, temp],'case')
        
ds = ds.assign_coords({"case": type})

# Calculate differences
diff = ds.isel(case=1)  - ds.isel(case=0) 
rel_diff = diff / ds.isel(case=1) 

### Quickplots of pSASU results & difference from pAD

In [ ]:
plt.figure(figsize=[14,14])
var = ['TOTECOSYSC' ,'TOTVEGC','TOTSOMC']
#var = ['GPP' ,'ELAI','ALT']
i = 1
for v in range(len(var)):
    plt.subplot(3, 2, i)
    ds[var[v]].isel(case=1).plot(robust=True)  
    plt.title("pSASU "+ var[v])
    plt.xlabel(None)
    plt.ylabel(None)
    i = i+1
    
    plt.subplot(3, 2, i)
    diff[var[v]].plot(robust=True) 
    plt.title("pSASU - pAD "+ var[v]) 
    plt.xlabel(None)
    plt.ylabel(None)   
    i = i+1


### lower soil C stocks at high latitues
- Look at depth profiles of C pools for a few points (also hard coded for now)

In [ ]:
plt.figure(figsize=[10,5])
var = 'SOM_PAS_C_vr'
plt.subplot(1, 4, 1)
ds[var].sel(lon=300,lat=-10, method='nearest').plot(hue='case',y='levsoi') ;
plt.gca().invert_yaxis() ;
plt.title('Tropical')
plt.ylabel('depth (m)')
plt.xscale('log',base=10) 
#plt.ylim(6,0)


plt.subplot(1, 4, 2)
ds[var].sel(lon=25,lat=50, method='nearest').plot(hue='case',y='levsoi') ;
plt.gca().invert_yaxis() ;
plt.title('Temperate')
plt.ylabel(None)
plt.xscale('log',base=10) 
#plt.ylim(6,0)

plt.subplot(1, 4, 3)
ds[var].sel(lon=155,lat=66, method='nearest').plot(hue='case',y='levsoi') ;
plt.gca().invert_yaxis() ;
plt.title('Arctic')
plt.ylabel(None)
plt.xscale('log',base=10) 
#plt.ylim(6,0)


plt.subplot(1, 4, 4)
ds[var].sel(lon=155,lat=66, method='nearest').plot(hue='case',y='levsoi') ;
plt.gca().invert_yaxis() ;
plt.title('Arctic')
plt.ylabel(None)
#plt.ylim(6,0)

### Big blob of passive C with depth in pAD